<a href="https://colab.research.google.com/github/giovxna/educa.lab/blob/main/educa_lab_Imers%C3%A3o_IA_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-genai google-adk

In [ ]:
import os
import textwrap
import warnings
from datetime import date
from google.colab import userdata
from IPython.display import display, Markdown
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types

In [ ]:
# Configuração da API
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
client = genai.Client()

In [ ]:
warnings.filterwarnings("ignore")
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Função auxiliar para executar agente

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response

In [15]:
def criar_agente_explicador(modo_usuario):
    instrucoes = {
        "infantil": "Explique como se o usuário tivesse 5 anos. Use analogias com brinquedos, animais e situações do cotidiano de uma criança.",
        "ensino_medio": "Explique como para um estudante do ensino médio, com exemplos aplicados, mas sem termos muito técnicos.",
        "adulto": "Explique de forma clara, objetiva e técnica, como para um adulto curioso.",
        "fofoca": "Explique como se fosse uma fofoca entre amigos. Use humor, gírias leves e analogias engraçadas como se estivesse contando um babado.",
        "memes": "Explique com tom de meme e linguagem da internet. Use emojis (mas de forma moderada), hashtags e expressões engraçadas."
    }

    return Agent(
        name="agente_explicador",
        model=MODEL_ID,
        instruction=instrucoes.get(modo_usuario, "Explique de forma clara e divertida."),
        description="Agente que explica conceitos com o estilo desejado."
    )

# Agente de quiz
def agente_quiz(assunto):
    quiz = Agent(
        name="agente_quiz",
        model=MODEL_ID,
        instruction="Crie uma pergunta ou desafio simples para testar se o usuário aprendeu o conteúdo.",
        description="Agente que cria um quiz ou desafio."
    )
    return call_agent(quiz, f"Crie uma pergunta para testar o conhecimento sobre: {assunto}")

# Agente de sugestão de curso
def agente_link_alura(assunto):
    link = Agent(
        name="agente_alura",
        model=MODEL_ID,
        instruction="Sugira um curso da Alura que aprofunde esse tema.",
        description="Agente que sugere cursos da Alura."
    )
    return call_agent(link, f"Sugira um curso da Alura para aprender mais sobre: {assunto}")

# ✅ Novo agente: plano de estudo
def agente_plano_estudo(assunto):
    agente = Agent(
        name="agente_plano_estudo",
        model=MODEL_ID,
        instruction=(
            "Crie um plano de estudo básico para quem quer aprender sobre esse assunto. "
            "Inclua temas complementares, sugestões de tempo e ordem de aprendizado."
        ),
        description="Gera um plano de estudos simples para guiar o usuário."
    )
    return call_agent(agente, f"Crie um plano de estudos para: {assunto}")

In [16]:
def aprofundar_assunto(explicador, assunto):
    while True:
        print("\n🔍 O que deseja fazer agora?")
        print("1 - Fazer uma nova pergunta sobre esse tema ❓")
        print("2 - Receber uma explicação mais profunda 🧠")
        print("3 - Tentar um desafio mais difícil sobre isso 🎯")
        print("4 - Ver outro assunto 🔁")
        print("5 - Encerrar 🚪")

        escolha = input("Escolha uma opção (1-5): ").strip()

        if escolha == "1":
            pergunta = input("Digite sua nova dúvida: ")
            prompt = f"Continuando sobre '{assunto}': {pergunta}"
            resposta = call_agent(explicador, prompt)
            print("\n📌 Resposta personalizada:\n")
            display(to_markdown(resposta))

        elif escolha == "2":
            prompt = f"Explique o conceito de '{assunto}' com mais profundidade e exemplos avançados."
            explicacao = call_agent(explicador, prompt)
            print("\n🔎 Explicação mais avançada:\n")
            display(to_markdown(explicacao))

        elif escolha == "3":
            quiz_dificil = Agent(
                name="quiz_dificil",
                model=MODEL_ID,
                instruction="Crie um desafio mais difícil, como um quiz com raciocínio ou aplicação prática.",
                description="Quiz avançado"
            )
            desafio = call_agent(quiz_dificil, f"Crie um desafio difícil sobre: {assunto}")
            print("\n🧩 Desafio avançado:\n")
            display(to_markdown(desafio))

        elif escolha == "4":
            break

        elif escolha == "5":
            print("\n👋 Até logo, bons estudos!")
            exit()

        else:
            print("Opção inválida. Tente novamente.")


In [17]:
def educa_lab(assunto, tempo, modo):
    explicador = criar_agente_explicador(modo)
    prompt = f"Explique o conceito de '{assunto}' em no máximo {tempo} minutos."
    explicacao = call_agent(explicador, prompt)
    quiz = agente_quiz(assunto)
    link = agente_link_alura(assunto)
    plano = agente_plano_estudo(assunto)

    print("\n📚 Explicação personalizada:\n")
    display(to_markdown(explicacao))

    print("\n❓ Pergunta rápida para testar aprendizado:\n")
    display(to_markdown(quiz))

    print("\n📎 Para saber mais:\n")
    display(to_markdown(link))

    print("\n🗂️ Plano de estudos sugerido:\n")
    display(to_markdown(plano))

    aprofundar_assunto(explicador, assunto)

# Menu inicial
def iniciar_educa_lab():
    while True:
        assunto = input("\nQuero aprender sobre: ")
        tempo = input("Tempo que tenho (2, 5, 10, 15 minutos): ")

        print("\n🧠 Como você quer aprender?")
        print("1 - Como se eu tivesse 5 anos 👶")
        print("2 - Como estudante do ensino médio 🧑")
        print("3 - Como adulto curioso 👩‍🎓")
        print("4 - Com analogias e fofoca 🤪")
        print("5 - Em memes 😂")
        print("6 - Surpreenda-me com o estilo! 🎲")

        modo_map = {
            "1": "infantil",
            "2": "ensino_medio",
            "3": "adulto",
            "4": "fofoca",
            "5": "memes"
        }

        modo_escolhido = input("Escolha um número: ")

        if modo_escolhido == "6":
            import random
            modo = random.choice(list(modo_map.values()))
            print(f"✨ Estilo sorteado: {modo}")
        else:
            modo = modo_map.get(modo_escolhido, "adulto")

        educa_lab(assunto, tempo, modo)

        repetir = input("\n🔁 Quer aprender outro assunto? (s/n): ").lower()
        if repetir != 's':
            print("\nAté a próxima! 👋")
            break


In [ ]:
# Executar o app
iniciar_educa_lab()


Quero aprender sobre: rest
Tempo que tenho (2, 5, 10, 15 minutos): 15

🧠 Como você quer aprender?
1 - Como se eu tivesse 5 anos 👶
2 - Como estudante do ensino médio 🧑
3 - Como adulto curioso 👩‍🎓
4 - Com analogias e fofoca 🤪
5 - Em memes 😂
6 - Surpreenda-me com o estilo! 🎲
Escolha um número: 6
✨ Estilo sorteado: memes

📚 Explicação personalizada:



> E aí, beleza?! 😎 Preparado pra desvendar o mistério do REST? Relaxa que em 15 minutinhos a gente DOMINA essa parada! 🚀
> 
> **REST: A parada mais "suave na nave" da internet!** 🌌
> 
> REST (Representational State Transfer) não é tipo uma receita de bolo, saca? É mais um *estilo arquitetural* pra galera que desenvolve sistemas distribuídos (tipo a internet toda!). Imagina que é um manual de boas práticas pra comunicação entre computadores, tipo um "guia de etiqueta" online. 💅
> 
> **Os 6 mandamentos do REST (pra não dar #fail):**
> 
> 1.  **Cliente-Servidor:** É tipo o iFood da vida. Você (cliente) faz o pedido e o restaurante (servidor) te entrega a comida. Cada um no seu quadrado, sem stress! 🍕
> 2.  **Sem Estado (Stateless):** Imagina que cada vez que você pede Ifood, tem que explicar tudo de novo. Ninguém merece! No REST, cada requisição tem que ter toda a info necessária, sem o servidor ficar lembrando de você. #vidaquesegue 🧘
> 3.  **Cacheável:** Pra que pedir a mesma pizza 3x seguidas? O REST incentiva a guardar as respostas (tipo um "print" da tela) pra não sobrecarregar o servidor. #esperto 🤓
> 4.  **Sistema em Camadas:** É tipo uma cebola! 🧅 Você só vê a camada de cima (a API), mas por baixo tem várias outras funcionando. Ninguém precisa saber da vida de todo mundo, né? #privacidade
> 5.  **Código sob Demanda (Opcional):** Tipo baixar um app novo pra ver um vídeo. O servidor pode te mandar um código pra rodar no seu computador, mas não é sempre que rola. #flexibilidade
> 6.  **Interface Uniforme:** ESSENCIAL! É a alma do negócio. Tem 4 sub-regras aqui, então presta atenção:
> 
>     *   **Identificação de Recursos:** Cada "coisa" (tipo um produto no iFood) tem um link único (URL).
>     *   **Manipulação de Recursos Através de Representações:** Você não "pega" o produto direto, você pega uma "foto" dele (JSON, XML, etc.).
>     *   **Auto Descritivo:** A "foto" do produto já te diz o que fazer com ele (editar, apagar, etc.).
>     *   **Hypermedia as the Engine of Application State (HATEOAS):** A API te fala quais são as próximas ações possíveis. Tipo um cardápio que te mostra o que você pode pedir.
> 
> **Na prática, como fica?** 🤔
> 
> *   Você usa os métodos HTTP (GET, POST, PUT, DELETE) pra fazer as operações. Tipo:
>     *   `GET /produtos/123` (pegar o produto com ID 123)
>     *   `POST /produtos` (criar um produto novo)
>     *   `PUT /produtos/123` (atualizar o produto 123)
>     *   `DELETE /produtos/123` (apagar o produto 123)
> 
> **E por que REST é tão legal?** ✨
> 
> *   **Escalabilidade:** Aguenta o tranco de muita gente usando ao mesmo tempo.
> *   **Simplicidade:** É fácil de entender e usar (depois dessa explicação, né?). 😉
> *   **Flexibilidade:** Funciona com várias linguagens e tecnologias.
> 
> **Resumindo:** REST é tipo um "manual de sobrevivência" pra web, que garante que todo mundo se entenda e a internet continue rodando liso. 😎
> 
> #REST #APIs #DesenvolvimentoWeb #TamoJunto #InternetSuave
> 



❓ Pergunta rápida para testar aprendizado:



> Ótimo! Para testar o conhecimento sobre REST, aqui vai uma pergunta:
> 
> **Pergunta:**
> 
> Qual das seguintes opções **NÃO** é uma característica fundamental de uma arquitetura RESTful?
> 
> a) Stateless (Sem estado)
> b) Cacheable (Cacheável)
> c) Stateful (Com estado)
> d) Uniform Interface (Interface Uniforme)
> 
> Essa pergunta testa a compreensão dos princípios básicos do REST, focando em identificar qual característica não se alinha com o estilo arquitetural.
> 



📎 Para saber mais:



> Para aprofundar seus conhecimentos sobre REST, sugiro o curso **"REST API: Conceitos e construindo uma API"** da Alura.
> 
> Nesse curso, você irá aprender:
> 
> *   **O que é REST:** Entenda os princípios e conceitos por trás da arquitetura REST.
> *   **Verbos HTTP:** Domine os métodos HTTP (GET, POST, PUT, DELETE) e como utilizá-los corretamente em uma API RESTful.
> *   **Códigos de Status:** Aprenda sobre os códigos de status HTTP e como eles indicam o resultado de uma requisição.
> *   **Formato de Dados:** Explore diferentes formatos de dados (JSON, XML) e como utilizá-los na troca de informações entre cliente e servidor.
> *   **Construção de uma API:** Coloque a mão na massa e construa uma API RESTful do zero, utilizando uma linguagem de programação (geralmente Java ou Node.js).
> *   **Boas Práticas:** Descubra as melhores práticas para projetar e implementar APIs RESTful eficientes e escaláveis.
> 
> Este curso é um excelente ponto de partida para quem deseja dominar a arquitetura REST e construir APIs de alta qualidade.
> 
> Para complementar seus estudos, você também pode procurar por outros cursos na Alura sobre tópicos relacionados, como:
> 
> *   **APIs:** Cursos sobre diferentes tipos de APIs (SOAP, GraphQL) e como integrá-las em seus projetos.
> *   **Microsserviços:** Cursos sobre arquitetura de microsserviços e como construir sistemas distribuídos e escaláveis.
> *   **Design de APIs:** Cursos sobre como projetar APIs intuitivas, fáceis de usar e que atendam às necessidades dos seus usuários.
> 
> Espero que esta sugestão seja útil!



🗂️ Plano de estudos sugerido:



> ## Plano de Estudos Básico para REST
> 
> Este plano de estudos tem como objetivo fornecer uma base sólida para entender e trabalhar com a arquitetura REST. Ele inclui tópicos complementares importantes e sugestões de tempo para cada área.
> 
> **Público-alvo:** Iniciantes com pouco ou nenhum conhecimento em REST.
> 
> **Duração total estimada:** 4-6 semanas (dependendo da sua dedicação e tempo disponível)
> 
> **Estrutura:**
> 
> **Semana 1: Fundamentos da Web e Protocolo HTTP**
> 
> *   **Objetivo:** Entender como a web funciona e como os dados são transferidos.
> *   **Tópicos:**
>     *   **Internet e a World Wide Web (WWW):**
>         *   O que é a internet?
>         *   O que é a web?
>         *   Como os navegadores funcionam?
>     *   **Protocolo HTTP:**
>         *   O que é um protocolo?
>         *   Estrutura de uma requisição HTTP (método, headers, body)
>         *   Estrutura de uma resposta HTTP (status code, headers, body)
>         *   Métodos HTTP principais: GET, POST, PUT, DELETE
>         *   Códigos de status HTTP importantes: 200, 201, 400, 401, 403, 404, 500
>     *   **URLs e URIs:**
>         *   O que são URLs e URIs?
>         *   Estrutura de uma URL
>     *   **HTML e Formulários:** (Conhecimento básico, não precisa ser especialista)
>         *   O que é HTML?
>         *   Como os formulários HTML funcionam?
> *   **Tempo estimado:** 5-7 horas
> *   **Recursos:**
>     *   Documentação HTTP da Mozilla Developer Network (MDN): [https://developer.mozilla.org/en-US/docs/Web/HTTP](https://developer.mozilla.org/en-US/docs/Web/HTTP)
>     *   Artigos introdutórios sobre a web (ex: "Como a Internet Funciona" do Tecnoblog)
>     *   Vídeos explicativos sobre HTTP no YouTube
> 
> **Semana 2: Introdução à Arquitetura REST**
> 
> *   **Objetivo:** Compreender os princípios e conceitos fundamentais da arquitetura REST.
> *   **Tópicos:**
>     *   **O que é REST?**
>         *   Representational State Transfer (REST)
>         *   Arquitetura vs. Protocolo
>     *   **Princípios REST:**
>         *   Client-Server
>         *   Stateless
>         *   Cacheable
>         *   Layered System
>         *   Code on Demand (opcional)
>         *   Uniform Interface (o mais importante)
>             *   Identification of Resources
>             *   Manipulation of Resources Through Representations
>             *   Self-descriptive Messages
>             *   Hypermedia as the Engine of Application State (HATEOAS)
>     *   **Recursos:**
>         *   O que são recursos em REST?
>         *   Como identificar recursos usando URIs?
>         *   Representações de recursos (JSON, XML)
> *   **Tempo estimado:** 5-7 horas
> *   **Recursos:**
>     *   Artigos e tutoriais sobre REST (ex: da Digital Ocean, da FreeCodeCamp)
>     *   Vídeos explicativos sobre REST no YouTube
>     *   Leitura do artigo original de Roy Fielding sobre REST (opcional, mais técnico): [https://www.ics.uci.edu/~fielding/pubs/dissertation/top.htm](https://www.ics.uci.edu/~fielding/pubs/dissertation/top.htm)
> 
> **Semana 3: RESTful APIs e JSON**
> 
> *   **Objetivo:** Aprender a projetar e consumir APIs RESTful, com foco em JSON como formato de dados.
> *   **Tópicos:**
>     *   **O que é uma API RESTful?**
>         *   Design de APIs RESTful
>         *   Boas práticas para nomear URIs (ex: usar substantivos, não verbos)
>         *   Uso correto dos métodos HTTP (GET, POST, PUT, DELETE)
>     *   **JSON (JavaScript Object Notation):**
>         *   Sintaxe do JSON
>         *   Tipos de dados em JSON (string, number, boolean, array, object, null)
>         *   Serialização e desserialização de JSON
>     *   **Ferramentas para testar APIs REST:**
>         *   Postman
>         *   Insomnia
>         *   curl (linha de comando)
> *   **Tempo estimado:** 6-8 horas
> *   **Recursos:**
>     *   Documentação do Postman/Insomnia
>     *   Tutoriais sobre JSON (ex: da MDN)
>     *   APIs REST públicas para praticar (ex: JSONPlaceholder)
>     *   Exercícios práticos: crie e consuma APIs RESTful simples usando as ferramentas aprendidas.
> 
> **Semana 4: HATEOAS e Segurança em APIs REST**
> 
> *   **Objetivo:** Aprofundar o conhecimento em REST com HATEOAS e entender os principais aspectos de segurança.
> *   **Tópicos:**
>     *   **HATEOAS (Hypermedia as the Engine of Application State):**
>         *   O que é HATEOAS e por que é importante?
>         *   Como implementar HATEOAS em APIs RESTful?
>         *   Exemplos de HATEOAS (links em respostas JSON)
>     *   **Segurança em APIs REST:**
>         *   Autenticação (verificação de identidade):
>             *   Basic Authentication
>             *   API Keys
>             *   OAuth 2.0
>             *   JWT (JSON Web Tokens)
>         *   Autorização (controle de acesso):
>             *   Roles e Permissions
>         *   HTTPS (criptografia da comunicação)
>         *   Validação de dados (para prevenir injeção de SQL e outros ataques)
>     *   **Versionamento de APIs:**
>         *   Por que versionar APIs?
>         *   Estratégias de versionamento (ex: na URL, nos headers)
> *   **Tempo estimado:** 6-8 horas
> *   **Recursos:**
>     *   Artigos e tutoriais sobre HATEOAS
>     *   Documentação sobre OAuth 2.0 e JWT
>     *   Guias de segurança para APIs REST (ex: OWASP API Security Top 10)
> 
> **Tópicos Complementares (Estude conforme o seu interesse e necessidade):**
> 
> *   **GraphQL:** Uma alternativa ao REST para APIs.
> *   **gRPC:** Um framework para chamadas de procedimento remoto (RPC) de alta performance.
> *   **WebSockets:** Para comunicação bidirecional em tempo real.
> *   **Cache:** Estratégias de cache para melhorar o desempenho de APIs REST.
> *   **Documentação de APIs (Swagger/OpenAPI):** Como documentar suas APIs de forma padronizada.
> 
> **Recomendações Gerais:**
> 
> *   **Pratique!** A teoria é importante, mas a prática é fundamental. Crie pequenas APIs RESTful para consolidar o que você aprendeu.
> *   **Use ferramentas.** Familiarize-se com ferramentas como Postman/Insomnia para testar suas APIs.
> *   **Leia código de outros.** Analise APIs RESTful existentes para aprender com exemplos reais.
> *   **Mantenha-se atualizado.** O mundo da tecnologia está sempre evoluindo. Acompanhe as novidades e tendências em APIs REST.
> 
> Este é um plano de estudos básico. Adapte-o às suas necessidades e interesses. Boa sorte!
> 



🔍 O que deseja fazer agora?
1 - Fazer uma nova pergunta sobre esse tema ❓
2 - Receber uma explicação mais profunda 🧠
3 - Tentar um desafio mais difícil sobre isso 🎯
4 - Ver outro assunto 🔁
5 - Encerrar 🚪
